In [1]:
import time
from icecream import ic
import requests,json 
from bs4 import BeautifulSoup       

In [2]:
headers = {
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.75 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'Referer': 'https://xyq.cbg.163.com/',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'zh-CN,zh;q=0.9',
}

def get_cookies(server_id):
    #all the following steps are to get a valid cookies
    session = requests.Session()
    session.get("https://xyq.cbg.163.com", headers=headers,)
    data={
          "act": "do_anon_auth",
          "server_id": server_id,
    }
    session.post("https://xyq.cbg.163.com/cgi-bin/login.py", data=data, headers=headers)
    return session.cookies 

cookies = get_cookies(79)

In [3]:
from pycbg.xyq.parser import Parser
from pycbg.xyq.object.xyqObject import *
p = Parser()

In [4]:
role_urls = []
for page in range(1):
    params = (
    ('act', 'search_role'),
    ('page', page),
    ('query_order', 'collect_num DESC')
)

    response = requests.get('https://xyq.cbg.163.com/cgi-bin/query.py', headers=headers, params=params, cookies=cookies)
    response.encoding = "GBK"
    bsobj = BeautifulSoup(response.text, "html.parser")
    role_info = bsobj.findAll("a",{"class":"soldImg"})
    role_urls.extend([info["href"] for info in role_info]) 

In [5]:
role_list = []
role_desc_list = []    
for index,url in enumerate(role_urls[:]):
    print(index)
    print(url)
    response = requests.get(url, cookies=cookies)
    print(response)
    response.encoding = "GBK"
    bsobj = BeautifulSoup(response.text, "html.parser")
    role_desc = bsobj.findAll("textarea", {"id":"equip_desc_value"})[0].text
    role_dict = eval(p.desc_proprocess(role_desc))
    role_desc_list.append(role_desc)
    role=p.parse_role(role_desc)
    role_list.append(role)

0
https://xyq.cbg.163.com/equip?s=79&eid=201807282000113-79-J872QGAUCAJZJ&equip_refer=33&view_loc=search_cond
<Response [200]>
1
https://xyq.cbg.163.com/equip?s=79&eid=201807261900113-79-7VDT6VPQJE1ZA&equip_refer=33&view_loc=search_cond
<Response [200]>
2
https://xyq.cbg.163.com/equip?s=79&eid=201806231800113-79-VTGMYJKDV5LQQ&equip_refer=33&view_loc=search_cond
<Response [200]>
3
https://xyq.cbg.163.com/equip?s=79&eid=201806080000113-79-QOOVLIRIFMYB1&equip_refer=33&view_loc=search_cond
<Response [200]>
4
https://xyq.cbg.163.com/equip?s=79&eid=201808202100113-79-ZTCO3GZDR6ZBJ&equip_refer=33&view_loc=search_cond
<Response [200]>
5
https://xyq.cbg.163.com/equip?s=79&eid=201807241900113-79-9SCSFKMVWQLAS&equip_refer=33&view_loc=search_cond
<Response [200]>
6
https://xyq.cbg.163.com/equip?s=79&eid=201706171600113-79-7XXFOVVUP9UC2&equip_refer=33&view_loc=search_cond
<Response [200]>
17320
7
https://xyq.cbg.163.com/equip?s=79&eid=201807211900113-79-YDMSEDPBGFPOA&equip_refer=33&view_loc=search_

In [6]:
for equip in role.equip_list:
    print(equip.__dict__)

{'attrs_list': [['等级', 140], ['抵抗封印等级', 36], ['格挡值', 34], ['气血', 114], ['防御', 28]], 'durance': 437, 'failure': 1, 'gemLevel': 8, 'typeID': 27205, 'lock': 1, 'lockNew': 0}
{'level': 115, 'DMG': 59, 'DEF': 107, 'loc': 3, 'durance': 282, 'suitSkill': '高级吸血', 'gemType': '气血', 'gemLevel': 7, 'typeID': 9112, 'lock': 0, 'lockNew': 9}
{'attrs_list': [['等级', 140], ['伤害', 36], ['法术暴击等级', 15], ['法术伤害', 14], ['法术伤害', 18]], 'durance': 34, 'failure': 1, 'gemLevel': 8, 'typeID': 27005, 'lock': 1, 'lockNew': 9}
{'attrs_list': [['等级', 140], ['法术伤害', 33], ['伤害', 16], ['法术伤害', 20], ['法术暴击等级', 21]], 'durance': 197, 'gemLevel': 7, 'typeID': 27105, 'lock': 1, 'lockNew': 0}
{'gemType': ['舍利子'], 'specialEffect': [], 'meltDict': {}, 'level': 140, 'DEF': 297, 'durance': 286, 'failure': 2, 'gemLevel': 14, 'suitSkill': '变身术之星灵仙子', 'hole': 5, 'totemCombo': '隔山打牛', 'creator': '雨中燃烧的泪强化打造', 'typeID': 2656, 'lock': 1, 'lockNew': 0}
{'gemType': ['太阳石'], 'specialEffect': ['天宫专用'], 'meltDict': {}, 'level': 160, 'accurat

In [7]:
for pet in role.pet_list:
    print(pet.__dict__)

{'skillList': ['573', '578', '424', '661', '428', '577', '403'], 'coreDict': {907: 5, 928: 1, 905: 5, 906: 5, 936: 5, 904: 5}, 'equip1': <pycbg.xyq.object.xyqObject.Equipment_pet object at 0x00000198C5D042B0>, 'equip2': <pycbg.xyq.object.xyqObject.Equipment_pet object at 0x00000198C5D04048>, 'equip3': <pycbg.xyq.object.xyqObject.Equipment_pet object at 0x00000198C5D041D0>, 'equip4': <pycbg.xyq.object.xyqObject.Equipment_pet object at 0x00000198C5D04080>, 'lock': 0, 'lockNew': 0, 'typeID': 102246, 'level': 180, 'DRNC': 4688, 'summon_color': 1, 'realColor': 5, 'APSkill': 428, 'AD_APTTD': 970, 'DEF_APTTD': 1324, 'raw_growth': 1272, 'growth': 1.272, 'MP_APTTD': 2015, 'HP_APTTD': 3550, 'SPD_APTTD': 765, 'MS_APTTD': 1086, 'used_yuanxiao': 10, 'used_ruyidan': 0, 'used_qianjinlu': 10, 'used_lianshou': 5, 'used_shuijinggao': 0, 'HP_P': 474, 'MP_P': 1038, 'AD_P': 212, 'DEF_P': 207, 'SPD_P': 158, 'unallocated_P': 0, 'lingxing': 101, 'tmp_lingxing': 101, 'jinjie_cd': 0, 'left_qlxl': 1, 'coreName':

In [8]:
print(len(role_list))

15
